In [2]:
#Importing the required libraries
import pandas as pd

In [3]:
df = pd.read_csv('spotify_millsongdata.csv')

In [4]:
df.shape

(57650, 4)

In [5]:
df.isna().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
#Drop the link column
df = df.sample(5000).drop('link',axis = 1).reset_index(drop=True)

In [8]:
df.shape

(5000, 3)

Text Column Cleaning

In [9]:
df['text'] = df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex = True)

In [10]:
df['text']

0       i told you  \r you'd have to leave  \r and you...
1       annie christian wanted to be number 1  \r but ...
2       i forget what to ask for,  \r there isn't anyt...
3       jean, jean, roses are red  \r all the leaves h...
4       on my back i lay it down for plenty  \r these ...
                              ...                        
4995    ice machine  \r running through my head secret...
4996    glory hallujah we are marching on  \r (as we s...
4997    six a.m. came way to early  \r but friday mean...
4998    farewell  \r to all of the heartaches you brou...
4999    canned laughter for applause  \r you've got op...
Name: text, Length: 5000, dtype: object

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajnishborkar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [15]:
stemmer = PorterStemmer()

In [16]:
#tokenization of the words
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(x) for x in token]
    return " ".join(a)

In [17]:
token('you are beautiful')

'you are beauti'

In [18]:
#Apply Toakenization in ou dataset
df['text'].apply(lambda x :token(x))

0       i told you you 'd have to leav and you listen ...
1       anni christian want to be number 1 but her kin...
2       i forget what to ask for , there is n't anyth ...
3       jean , jean , rose are red all the leav have g...
4       on my back i lay it down for plenti these bone...
                              ...                        
4995    ice machin run through my head secretli the sh...
4996    glori hallujah we are march on ( as we sing th...
4997    six a.m. came way to earli but friday mean it ...
4998    farewel to all of the heartach you brought so ...
4999    can laughter for applaus you 've got open door...
Name: text, Length: 5000, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [20]:
#Instance of vectorization
tfid = TfidfVectorizer(analyzer='word',stop_words='english')

In [21]:
#create a matrics of the words
metrix = tfid.fit_transform(df['text'])

In [22]:
# create the array of similarity
similarity = cosine_similarity(metrix)

In [23]:
similarity[0]

array([1.        , 0.00323378, 0.00896721, ..., 0.01567963, 0.00196547,
       0.0308233 ])

In [24]:
def recommender(song_name):
    idx = df[df['song'] == song_name].index[0]
    distance = sorted(list(enumerate(similarity[idx])),reverse=True,key = lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [25]:
df

,artist,song,text
0,Santana,You Just Don't Care,i told you \r you'd have to leave \r and you...
1,Prince,Annie Christian,annie christian wanted to be number 1 \r but ...
2,James Taylor,Carry Me On My Way,"i forget what to ask for, \r there isn't anyt..."
3,Oliver,Jean,"jean, jean, roses are red \r all the leaves h..."
4,Queen Adreena,Wolverines,on my back i lay it down for plenty \r these ...
...,...,...,...
4995,Depeche Mode,Ice Machine,ice machine \r running through my head secret...
4996,Hank Snow,Gloryland March,glory hallujah we are marching on \r (as we s...
4997,Leann Rimes,I Got It Bad,six a.m. came way to early \r but friday mean...
4998,Neil Diamond,"Farewell, So Long, Goodbye",farewell \r to all of the heartaches you brou...


In [26]:
recommender('Famous Last Words')

['Famous Groupies', 'More Than Words', 'Year Of The Knife', 'Lost For Words']

In [27]:
import pickle

In [28]:
pickle.dump(similarity,open('similarity','wb'))

In [29]:
pickle.dump(df,open('df','wb'))

In [30]:
pickle.dump(similarity,open('similarity.pkl','rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'similarity.pkl'